# **Homework 2**
**Name:** Amirhossein Jamali

**GitHub repository:** https://github.com/jamir91/NLP

In [1]:
from __future__ import unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bidi.algorithm import get_display
from arabic_reshaper import reshape
from hazm import Normalizer, Stemmer, Lemmatizer, word_tokenize

In [2]:
class DataPreprocessor:
#     train_data = pd.DataFrame()
#     test_data = pd.DataFrame()

    train_data = pd.read_pickle('test_dataframe')
    test_data = pd.read_pickle('test_dataframe')

    stop_words = list()
    n_tokens = 0
    frequencies = {}
    wordToIndex = {}
    indexToWord = {}
    index = 0

    def __init__(self, address='./data/'):
#         self.read_data(address)
        self.plot_distribution()
#         self.clean_text()
        self.count_words()
        self.func()

    def read_data(self, address):
        self.train_data = pd.read_csv(address + 'train.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')
        self.train_data.drop(self.train_data[self.train_data.category=='category'].index, axis=0, inplace=True)
        self.train_data.dropna(subset=['text'], inplace=True)
        self.test_data = pd.read_csv(address + 'test.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')
        self.test_data.drop(self.test_data[self.test_data.category=='category'].index, axis=0, inplace=True)
        self.test_data.dropna(subset=['text'], inplace=True)
        with open(address + 'Stop_words.txt', encoding="utf8") as f:
            self.stop_words = f.read().splitlines()
        norm = Normalizer()
        self.stop_words = [norm.normalize(i) for i in self.stop_words]
#         print(self.stop_words)
#         print('-------------------------------------------------------------')
#         print(self.train_data.info())
#         print('-------------------------------------------------------------')
#         print(self.test_data.info())

    def draw_plot(self, labels, counts, fname):
        font = {"family": "B Nazanin", "size": 15}
        plt.rc("font", **font)

        persian_labels = [get_display(reshape(label)) for label in labels]
        fig = plt.figure(figsize = (10, 5))
        plt.bar(persian_labels, counts, width = 0.7, color='darkblue')

        for index,data in enumerate(counts):
            plt.text(x=index, y =data+1, s=f"{data}", color='darkgreen', fontdict=dict(fontsize=13))

        plt.savefig(fname, dpi=200)

    def plot_distribution(self):
        labels, counts = np.unique(self.train_data['category'], return_counts=True)
        self.draw_plot(labels, counts, 'train.png')
        print('-------------------------------------------------------------')
        labels, counts = np.unique(self.test_data['category'], return_counts=True)
        self.draw_plot(labels, counts, 'test.png')
    
    def del_stop_words(self, words):
        return [word for word in words if word not in self.stop_words]
    
    def remove_none_alpha(self, d):
        persian_chars = u'\u200c ‌آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی۰۱۲۳۴۵۶۷۸۹.؟?0123456789'
        dd = ''
        d = d.replace('\n', ' ')
        for c in d:
            if c in persian_chars:
                dd += c

        return dd
    
    def replace_digits(self, d):
        persian_digits = u'0123456789۱۲۳۴۵۶۷۸۹۰'
        for c in d:
            if c in persian_digits:
                d = d.replace(c, 'N')
        while 'NN' in d:
            d = d.replace('NN', 'N')
        while 'N.N' in d:
            d = d.replace('N.N', 'N')

        return d
    
    def dfstemmer(self, tokens):
        stemmer = Stemmer()
        
        return [stemmer.stem(l) for l in tokens]

    def clean_text(self):
        self.train_data.drop(self.train_data[self.train_data.category=='category'].index, axis=0, inplace=True)
        self.train_data.dropna(subset=['text'], inplace=True)
        self.train_data = self.train_data[['category', 'text']]
        normalizer = Normalizer()
        self.train_data['text'] = self.train_data['text'].apply(normalizer.normalize)
        self.train_data['text'] = self.train_data['text'].apply(self.remove_none_alpha)
        self.train_data['text'] = self.train_data['text'].apply(self.replace_digits)
        self.train_data['text'] = self.train_data['text'].apply(word_tokenize)
        self.train_data['text'] = self.train_data['text'].apply(self.del_stop_words)
        self.train_data['text'] = self.train_data['text'].apply(self.dfstemmer)
        self.train_data.reset_index(drop=True, inplace=True)
        self.train_data.to_pickle('train_dataframe')
#         print(self.train_data['text'][1])
        
        self.test_data.drop(self.test_data[self.test_data.category=='category'].index, axis=0, inplace=True)
        self.test_data.dropna(subset=['text'], inplace=True)
        self.test_data = self.test_data[['category', 'text']]
        self.test_data['text'] = self.test_data['text'].apply(normalizer.normalize)
        self.test_data['text'] = self.test_data['text'].apply(self.remove_none_alpha)
        self.test_data['text'] = self.test_data['text'].apply(self.replace_digits)
        self.test_data['text'] = self.test_data['text'].apply(word_tokenize)
        self.test_data['text'] = self.test_data['text'].apply(self.del_stop_words)
        self.test_data['text'] = self.test_data['text'].apply(self.dfstemmer)
        self.test_data.reset_index(drop=True, inplace=True)
        self.test_data.to_pickle('test_dataframe')
#         print(self.test_data['text'][1])

    def save_dict(self, d, n=200):
        f_out = open("frequent.txt", "w", encoding="utf-8")
        for item, count in list(d.items())[:n]:
            f_out.write(f'{item}, {count}\n')

    def freq(self, tokens):
        self.index += 1
        print(self.index)
        for t in tokens:
            self.n_tokens += 1
            if t in self.frequencies:
                self.frequencies[t] += 1
            else:
                self.frequencies[t] = 1
        
        self.frequencies = dict(sorted(self.frequencies.items(), key=lambda item: item[1], reverse=True))
        self.save_dict(self.frequencies)

    def count_words(self):
        self.train_data['text'].map(self.freq)
        self.frequencies = dict(sorted(self.frequencies.items(), key=lambda item: item[1], reverse=True))
        self.save_dict(self.frequencies)

        
    def func(self):
        print(self.n_tokens)
#         print(self.frequencies)

    def map_word_index(self):
        pass

    def tokenize(self):
        pass

In [3]:
class NaiveBayesClassifier:
    
    def __init__(self, address='./data/'):
        pass
    
    def calculate_word_log_prob_per_class(self):
        pass
    
    def calculate_log_prior(self):
        pass
    
    def predict_naive_bayes(self):
        pass
    
    def evaluate(self):
        pass

In [4]:
if __name__ == '__main__':
    dp = DataPreprocessor()

123456789012345678901234567890123456789012345678901234567890
1 start freq----------------------------------------------------------------------
2 start freq----------------------------------------------------------------------
3 start freq----------------------------------------------------------------------
4 start freq----------------------------------------------------------------------
5 start freq----------------------------------------------------------------------
6 start freq----------------------------------------------------------------------
7 start freq----------------------------------------------------------------------
8 start freq----------------------------------------------------------------------
9 start freq----------------------------------------------------------------------
10 start freq----------------------------------------------------------------------
11 start freq----------------------------------------------------------------------
12 start freq-----------

138 start freq----------------------------------------------------------------------
139 start freq----------------------------------------------------------------------
140 start freq----------------------------------------------------------------------
141 start freq----------------------------------------------------------------------
142 start freq----------------------------------------------------------------------
143 start freq----------------------------------------------------------------------
144 start freq----------------------------------------------------------------------
145 start freq----------------------------------------------------------------------
146 start freq----------------------------------------------------------------------
147 start freq----------------------------------------------------------------------
148 start freq----------------------------------------------------------------------
149 start freq---------------------------------------------------

274 start freq----------------------------------------------------------------------
275 start freq----------------------------------------------------------------------
276 start freq----------------------------------------------------------------------
277 start freq----------------------------------------------------------------------
278 start freq----------------------------------------------------------------------
279 start freq----------------------------------------------------------------------
280 start freq----------------------------------------------------------------------
281 start freq----------------------------------------------------------------------
282 start freq----------------------------------------------------------------------
283 start freq----------------------------------------------------------------------
284 start freq----------------------------------------------------------------------
285 start freq---------------------------------------------------

378 start freq----------------------------------------------------------------------
379 start freq----------------------------------------------------------------------
380 start freq----------------------------------------------------------------------
381 start freq----------------------------------------------------------------------
382 start freq----------------------------------------------------------------------
383 start freq----------------------------------------------------------------------
384 start freq----------------------------------------------------------------------
385 start freq----------------------------------------------------------------------
386 start freq----------------------------------------------------------------------
387 start freq----------------------------------------------------------------------
388 start freq----------------------------------------------------------------------
389 start freq---------------------------------------------------

498 start freq----------------------------------------------------------------------
499 start freq----------------------------------------------------------------------
500 start freq----------------------------------------------------------------------
501 start freq----------------------------------------------------------------------
502 start freq----------------------------------------------------------------------
503 start freq----------------------------------------------------------------------
504 start freq----------------------------------------------------------------------
505 start freq----------------------------------------------------------------------
506 start freq----------------------------------------------------------------------
507 start freq----------------------------------------------------------------------
508 start freq----------------------------------------------------------------------
509 start freq---------------------------------------------------

...

17042 start freq----------------------------------------------------------------------
17043 start freq----------------------------------------------------------------------
17044 start freq----------------------------------------------------------------------
17045 start freq----------------------------------------------------------------------
17046 start freq----------------------------------------------------------------------
17047 start freq----------------------------------------------------------------------
17048 start freq----------------------------------------------------------------------
17049 start freq----------------------------------------------------------------------
17050 start freq----------------------------------------------------------------------
17051 start freq----------------------------------------------------------------------
17052 start freq----------------------------------------------------------------------
17053 start freq---------------------------

17137 start freq----------------------------------------------------------------------
17138 start freq----------------------------------------------------------------------
17139 start freq----------------------------------------------------------------------
17140 start freq----------------------------------------------------------------------
17141 start freq----------------------------------------------------------------------
17142 start freq----------------------------------------------------------------------
17143 start freq----------------------------------------------------------------------
17144 start freq----------------------------------------------------------------------
17145 start freq----------------------------------------------------------------------
17146 start freq----------------------------------------------------------------------
17147 start freq----------------------------------------------------------------------
17148 start freq---------------------------

17232 start freq----------------------------------------------------------------------
17233 start freq----------------------------------------------------------------------
17234 start freq----------------------------------------------------------------------
17235 start freq----------------------------------------------------------------------
17236 start freq----------------------------------------------------------------------
17237 start freq----------------------------------------------------------------------
17238 start freq----------------------------------------------------------------------
17239 start freq----------------------------------------------------------------------
17240 start freq----------------------------------------------------------------------
17241 start freq----------------------------------------------------------------------
17242 start freq----------------------------------------------------------------------
17243 start freq---------------------------

17329 start freq----------------------------------------------------------------------
17330 start freq----------------------------------------------------------------------
17331 start freq----------------------------------------------------------------------
17332 start freq----------------------------------------------------------------------
17333 start freq----------------------------------------------------------------------
17334 start freq----------------------------------------------------------------------
17335 start freq----------------------------------------------------------------------
17336 start freq----------------------------------------------------------------------
17337 start freq----------------------------------------------------------------------
17338 start freq----------------------------------------------------------------------
17339 start freq----------------------------------------------------------------------
17340 start freq---------------------------

17428 start freq----------------------------------------------------------------------
17429 start freq----------------------------------------------------------------------
17430 start freq----------------------------------------------------------------------
17431 start freq----------------------------------------------------------------------
17432 start freq----------------------------------------------------------------------
17433 start freq----------------------------------------------------------------------
17434 start freq----------------------------------------------------------------------
17435 start freq----------------------------------------------------------------------
17436 start freq----------------------------------------------------------------------
17437 start freq----------------------------------------------------------------------
17438 start freq----------------------------------------------------------------------
17439 start freq---------------------------

In [ ]:
train_df = pd.read_pickle('train_dataframe')
test_df = pd.read_pickle('test_dataframe')
train_df

In [ ]:
from __future__ import unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bidi.algorithm import get_display
from arabic_reshaper import reshape
from hazm import Normalizer, Stemmer, Lemmatizer, word_tokenize

with open('./data/Stop_words.txt', encoding="utf8") as f:
    stop_words = f.read().splitlines()

normalizer = Normalizer()
stop_words = [normalizer.normalize(i) for i in stop_words]

train_data = pd.read_csv('./data/train.csv', sep='\t', error_bad_lines= False , encoding= 'utf-8')

def remove_none_alpha(d):
    persian_chars = u'\u200c ‌آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی۰۱۲۳۴۵۶۷۸۹.؟?0123456789'
    dd = ''
    d = d.replace('\n', ' ')
    for c in d:
        if c in persian_chars:
            dd += c

    return dd

def replace_digits(d):
    persian_digits = u'0123456789۱۲۳۴۵۶۷۸۹۰'
    for c in d:
        if c in persian_digits:
            d = d.replace(c, 'N')
    while 'NN' in d:
        d = d.replace('NN', 'N')
    while 'N.N' in d:
        d = d.replace('N.N', 'N')
    return d

In [ ]:

train_data.dropna(subset=['text'], inplace=True)
train_data = train_data[['category', 'text']]
train_data['text'] = train_data['text'].apply(normalizer.normalize)
train_data['text'] = train_data['text'].apply(remove_none_alpha)
train_data['text'] = train_data['text'].apply(replace_digits)
train_data['text'] = train_data['text'].apply(word_tokenize)


In [ ]:
train_data['text'][2]

In [ ]:
with open('./data/Stop_words.txt', encoding="utf8") as f:
    stop_words = f.read().splitlines()

In [ ]:
stop_words = [normalizer.normalize(i) for i in stop_words]
stop_words

In [ ]:
def del_stop_words(words, stopwords=stop_words):
    return [word for word in words if word not in stopwords]

In [ ]:
train_df['text'] = train_df['text'].apply(replace_digits)
train_df['text'] = train_df['text'].apply(normalizer.normalize)
train_df['text'] = train_df['text'].apply(word_tokenize)


In [ ]:
train_df['text'] = train_df['text'].apply(del_stop_words)
train_df['text'][1]

In [ ]:
def dfstemmer(L):
    return [stemmer.stem(l) for l in L]
# train_df['text'] = [stemmer.stem(l) for l in train_df.index]

def detokenized(L):
    return " ".join(L)

In [ ]:
train_df['text'] = train_df['text'].apply(dfstemmer)

In [ ]:
#train_df['text'] = train_df['text'].apply(detokenized)

In [ ]:
train_df

In [ ]:
frequencies = {}
n_tokens = 0

In [ ]:
train_df['text'].apply(freq)

In [ ]:
frequencies

In [ ]:
frequencies = dict(sorted(frequencies.items(), key=lambda item: item[1], reverse=True)[:])
frequencies

In [ ]:
train_df

In [ ]:
def sent_tokenizer(text):
    # sentence separators: ؟ . ! ?
    seperators = "؟.!?"
    pattern = r"[^{0}]+[{0}]".format(seperators)
    return re.findall(pattern, text)

In [ ]:
def word_tokenizer(sent):
    seperators = "؟!.?,:;، \t"
    pattern = r"([^{0}]+)([{0}])".format(seperators)

    words_sep = re.findall(pattern, sent)
    words = [w for word, sep in words_sep for w in [word, sep] if w != " "]
    return words

In [ ]:
import re
tokenized = [word_tokenizer(sent) for sent in sent_tokenizer(train_df['text'][200])]
tokenized[3]

In [ ]:
! pip install hazm

In [ ]:
from __future__ import unicode_literals
from hazm import Normalizer, Stemmer, Lemmatizer, word_tokenize

In [ ]:
word_tokenizer(sent) for sent in sent_tokenizer(train_df['text'][200])

In [ ]:
normalizer = Normalizer()
tokenized = normalizer.normalize(train_df['text'][200])
print(tokenized)
tokenized = word_tokenize(tokenized)
print(tokenized)
stemmer = Stemmer()
for i in tokenized:
    print(stemmer.stem(i))
    
tokenized = stemmer.stem(train_df['text'][200])

print(stemmer.stem(' من دستان'))

# print(tokenized)

In [ ]:
df=pd.DataFrame({'A':[['love'],
 ['amazing'],
 ['love'],
 ['sounds', 'good', 'great', 'price'],
 ['two',
  'hours',
  'wife',
  'asked',
  'turn',
  'even',
  'bose',
  'products',
  'sound',
  'better']]})

In [ ]:
df

In [ ]:
df=df.A.apply(pd.Series).T.unstack().reset_index().dropna().drop('level_1',1).set_index('level_0')
df.groupby([df.index.get_level_values(0),0]).size()

In [ ]:
df1 =pd.DataFrame( train_df['text'])
df1

In [ ]:
df1 = train_df.text.apply(pd.Series).T.unstack().reset_index().dropna().drop('level_1',1).set_index('level_0')
df1.groupby([df.index.get_level_values(0),0]).size()